In [1]:
from WElib import Walker, FunctionProgressCoordinator, FunctionStepper, StaticBinner, Recycler, SplitMerger

In [2]:
initial_state = [1.0]

def pc_func(state):
    return state[0]

def stepper_func(state):
    return [state[0] + 1.0]

In [3]:
n_walkers = 4
initial_states = [[1.0], [1.0], [2.0], [3.0]]
weight = 1.0 / n_walkers
walkers = [Walker(initial_state, weight) for initial_state in initial_states]
stepper = FunctionStepper(stepper_func)
walkers = stepper.run(walkers)
pc = FunctionProgressCoordinator(pc_func)
walkers = pc.run(walkers)
for w in walkers:
    print(w)
recycler = Recycler(2.0)
walkers = recycler.run(walkers)
for w in walkers:
    print(w)
print(recycler.flux)
print(recycler.recycled_walkers)
binner = StaticBinner([1.0, 2.0, 3.0, 4.0])
walkers = binner.run(walkers)
sm = SplitMerger(n_walkers)
walkers = sm.run(walkers)
for w in walkers:
    print(w)

<WElib.Walker weight 0.25, progress coordinate [2.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [2.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [3.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [4.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [2.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [2.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [3.0], bin assignment None>
<WElib.Walker weight 0.25, progress coordinate [4.0], bin assignment None>
0.5
[]
<WElib.Walker weight 0.125, progress coordinate [2.0], bin assignment 2>
<WElib.Walker weight 0.125, progress coordinate [2.0], bin assignment 2>
<WElib.Walker weight 0.125, progress coordinate [2.0], bin assignment 2>
<WElib.Walker weight 0.125, progress coordinate [2.0], bin assignment 2>
<WElib.Walker weight 0.0625, progress coordinate [3.0], bin assignment 3>
<WElib.Walker weight 0.0625

In [4]:
for w in walkers:
    print(w._initial_pcs)

[2.0]
[2.0]
[2.0]
[2.0]
[3.0]
[3.0]
[3.0]
[3.0]
[4.0]
[4.0]
[4.0]
[4.0]


In [5]:
w0 = walkers[0]
print(w0)

<WElib.Walker weight 0.125, progress coordinate [2.0], bin assignment 2>


In [6]:
print(w0._initial_state)

[2.0]


In [7]:
wtest = Walker([1.0], 0.3)
print(wtest.state, wtest._initial_state)
wtest.update([2.0])
print(wtest.state, wtest._initial_state)

[1.0] [1.0]
[2.0] [1.0]


In [8]:
wtest = stepper.run(wtest)[0]

In [9]:
print(wtest.state, wtest._initial_state)

[3.0] [1.0]
